In [1]:
import tensorflow as tf
import pathlib
from tensorflow.keras.applications.efficientnet import preprocess_input

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print("\n✅ Import menggunakan efficientnet preprocess_input")

TensorFlow version: 2.16.1
GPU Available: []

✅ Import menggunakan efficientnet preprocess_input


In [ ]:
# =============================================================================
# CELL 2: KONFIGURASI PATH DAN PARAMETER
# =============================================================================
# ❌ TIDAK ADA PERUBAHAN di cell ini - sama dengan versi ResNet50
# =============================================================================

base_dir = pathlib.Path(".")
data_split_dir = base_dir / 'dataset_final'

# Parameter gambar (sama untuk EfficientNet-B0 dan ResNet50)
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = 4

print("Konfigurasi:")
print(f"  - Image Size: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Num Classes: {NUM_CLASSES}")
print(f"  - Data Path: {data_split_dir}")

In [ ]:
# =============================================================================
# CELL 3: MEMUAT DATASET
# =============================================================================
# ❌ TIDAK ADA PERUBAHAN di cell ini - sama dengan versi ResNet50
# =============================================================================

print("="*60)
print("PROSES PEMUATAN DATASET")
print("="*60)

# 1. Load Training Dataset
print(f"\n[1/3] Memuat {data_split_dir / 'train'}")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'train',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

# 2. Load Validation Dataset
print(f"\n[2/3] Memuat {data_split_dir / 'val'}")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'val',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED
)

# 3. Load Test Dataset
print(f"\n[3/3] Memuat {data_split_dir / 'test'}")
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'test',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED
)

# Simpan nama kelas
class_names = train_ds.class_names

print(f"\n" + "="*60)
print("INFORMASI KELAS & MAPPING")
print("="*60)
print(f"Kelas ditemukan: {class_names}")
print(f"\n{'Index':<10} | {'Folder':<10} | {'Kandungan Aflatoksin'}")
print("-" * 50)
for idx, name in enumerate(class_names):
    print(f"{idx:<10} | {name:<10} | {name} PPB")

In [ ]:
# =============================================================================
# CELL 4: DATA AUGMENTATION
# =============================================================================
# ❌ TIDAK ADA PERUBAHAN di cell ini - TETAP SAMA dengan versi ResNet50
#
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║  🚫 PERINGATAN: JANGAN TAMBAHKAN AUGMENTASI BRIGHTNESS/CONTRAST!          ║
# ║                                                                           ║
# ║  Alasan:                                                                  ║
# ║  - Ground truth klasifikasi aflatoksin ditentukan oleh INTENSITAS CAHAYA  ║
# ║  - Jika brightness diubah, nilai PPB akan berubah:                        ║
# ║    • Brightness naik: 2 PPB → bisa terlihat seperti 3 atau 4 PPB          ║
# ║    • Brightness turun: 2 PPB → bisa terlihat seperti 1 PPB                ║
# ║  - Ini membuat model belajar dari data dengan LABEL YANG SALAH!           ║
# ║                                                                           ║
# ║  Augmentasi yang AMAN:                                                    ║
# ║  ✅ RandomFlip - orientasi tidak mempengaruhi nilai PPB                   ║
# ║  ✅ RandomRotation - sudut tidak mempengaruhi nilai PPB                   ║
# ║                                                                           ║
# ║  Augmentasi yang TIDAK BOLEH:                                             ║
# ║  ❌ RandomBrightness - MENGUBAH nilai PPB!                                ║
# ║  ❌ RandomContrast - MENGUBAH nilai PPB!                                  ║
# ║  ❌ ColorJitter - MENGUBAH nilai PPB!                                     ║
# ╚═══════════════════════════════════════════════════════════════════════════╝
# =============================================================================

print("Setting up data augmentation...")
print("\n" + "="*60)
print("⚠️  CATATAN PENTING TENTANG AUGMENTASI")
print("="*60)
print("""
Pada dataset aflatoksin, klasifikasi ditentukan oleh:
  • UKURAN area fluorescence
  • INTENSITAS KECERAHAN fluorescence

Oleh karena itu:
  ✅ RandomFlip      - AMAN (orientasi tidak mengubah nilai PPB)
  ✅ RandomRotation  - AMAN (sudut tidak mengubah nilai PPB)
  ❌ RandomBrightness - TIDAK BOLEH (mengubah intensitas = mengubah nilai PPB!)
  ❌ RandomContrast   - TIDAK BOLEH (mengubah kontras = mengubah nilai PPB!)
""")

# Augmentasi yang AMAN untuk dataset aflatoksin
data_augmentation = tf.keras.Sequential([
    # 1. Random Horizontal Flip - AMAN
    tf.keras.layers.RandomFlip("horizontal"),
    
    # 2. Random Rotation - AMAN
    # 0.028 radian ≈ 1.6 derajat (sama dengan versi ResNet50)
    tf.keras.layers.RandomRotation(0.028),
    
    # ❌ JANGAN TAMBAHKAN INI:
    # tf.keras.layers.RandomBrightness(0.1),  # TIDAK BOLEH!
    # tf.keras.layers.RandomContrast(0.1),    # TIDAK BOLEH!
    
], name="data_augmentation")

print("\nAugmentasi yang digunakan:")
print("  ✓ RandomFlip (horizontal)")
print("  ✓ RandomRotation (±1.6°)")
print("\nAugmentasi yang TIDAK digunakan (karena akan mengubah nilai PPB):")
print("  ✗ RandomBrightness")
print("  ✗ RandomContrast")

In [ ]:
# =============================================================================
# CELL 5: FUNGSI PREPROCESSING
# =============================================================================
#
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║  🔄 PERUBAHAN: Fungsi preprocess_input yang dipanggil berbeda             ║
# ║                                                                           ║
# ║  Meskipun KODE-nya SAMA, PERILAKU-nya BERBEDA karena:                     ║
# ║  - Di Cell 1, kita import dari 'efficientnet' bukan 'resnet50'            ║
# ║  - Jadi preprocess_input() di sini adalah versi EfficientNet              ║
# ║                                                                           ║
# ║  PERBEDAAN NORMALISASI:                                                   ║
# ║  ┌─────────────────────────────────────────────────────────────────────┐  ║
# ║  │ ResNet50 (mode "caffe"):                                            │  ║
# ║  │   1. Konversi RGB → BGR                                             │  ║
# ║  │   2. Kurangi mean ImageNet: [103.939, 116.779, 123.68]              │  ║
# ║  │   3. Output range: sekitar [-128, 128]                              │  ║
# ║  ├─────────────────────────────────────────────────────────────────────┤  ║
# ║  │ EfficientNet (mode "torch"):                                        │  ║
# ║  │   1. Bagi dengan 255 → range [0, 1]                                 │  ║
# ║  │   2. Normalize: (x - mean) / std                                    │  ║
# ║  │      mean = [0.485, 0.456, 0.406]                                   │  ║
# ║  │      std = [0.229, 0.224, 0.225]                                    │  ║
# ║  │   3. Output range: sekitar [-2.1, 2.6]                              │  ║
# ║  │   4. Tetap RGB (tidak dikonversi ke BGR)                            │  ║
# ║  └─────────────────────────────────────────────────────────────────────┘  ║
# ╚═══════════════════════════════════════════════════════════════════════════╝
# =============================================================================

def preprocess_for_efficientnet(images, labels, training=False):
    """
    Preprocessing untuk EfficientNet.
    
    CATATAN: Meskipun struktur kode sama dengan versi ResNet50,
    fungsi preprocess_input() yang dipanggil adalah versi EfficientNet
    (karena import di Cell 1).
    
    Args:
        images: Batch gambar dengan nilai pixel 0-255
        labels: One-hot encoded labels
        training: Boolean, True untuk training (apply augmentation)
    
    Returns:
        Tuple (preprocessed_images, labels)
    """
    # Cast ke float32
    images = tf.cast(images, tf.float32)
    
    # Terapkan augmentasi HANYA saat training
    # (hanya flip dan rotation - TIDAK ADA brightness/contrast)
    if training:
        images = data_augmentation(images, training=True)
    
    # ⬇️⬇️⬇️ INI YANG BERBEDA PERILAKUNYA ⬇️⬇️⬇️
    # Meskipun kode sama, fungsi ini adalah versi EfficientNet
    # karena import dari tensorflow.keras.applications.efficientnet
    images = preprocess_input(images)
    # ⬆️⬆️⬆️ INI YANG BERBEDA PERILAKUNYA ⬆️⬆️⬆️
    
    return images, labels

print("Fungsi preprocessing untuk EfficientNet telah dibuat.")
print("\n" + "="*60)
print("PERBEDAAN NORMALISASI")
print("="*60)
print("""
ResNet50 (mode "caffe"):
  • RGB → BGR
  • Subtract mean [103.939, 116.779, 123.68]
  • Output: ~[-128, 128]

EfficientNet (mode "torch"):
  • Tetap RGB
  • Normalize: (x/255 - mean) / std
  • Output: ~[-2.1, 2.6]
""")

In [ ]:
# =============================================================================
# CELL 6: TERAPKAN PREPROCESSING & OPTIMASI PIPELINE
# =============================================================================
#
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║  🔄 PERUBAHAN: Menambahkan .cache() dan .prefetch()                       ║
# ║                                                                           ║
# ║  SEBELUM (ResNet50):                                                      ║
# ║  train_ds = train_ds.map(...)                                             ║
# ║                                                                           ║
# ║  SESUDAH (EfficientNet):                                                  ║
# ║  train_ds = train_ds.map(...).cache().prefetch(...)                       ║
# ║                                                                           ║
# ║  PENJELASAN:                                                              ║
# ║  • .cache() = simpan data di memory setelah load pertama                  ║
# ║               Epoch 2+ akan lebih cepat karena tidak perlu baca disk      ║
# ║  • .prefetch() = load batch berikutnya sementara GPU proses batch saat ini║
# ║                  Menghilangkan bottleneck I/O                             ║
# ╚═══════════════════════════════════════════════════════════════════════════╝
# =============================================================================

print("Menerapkan preprocessing ke dataset...")

# Training Dataset
# ⬇️ PERUBAHAN: Menambahkan .cache().prefetch() ⬇️
train_ds = train_ds.map(
    lambda x, y: preprocess_for_efficientnet(x, y, training=True),
    num_parallel_calls=AUTOTUNE
).cache().prefetch(buffer_size=AUTOTUNE)

print("✓ Training dataset: augmentation ON, cache ON, prefetch ON")

# Validation Dataset
# ⬇️ PERUBAHAN: Menambahkan .cache().prefetch() ⬇️
val_ds = val_ds.map(
    lambda x, y: preprocess_for_efficientnet(x, y, training=False),
    num_parallel_calls=AUTOTUNE
).cache().prefetch(buffer_size=AUTOTUNE)

print("✓ Validation dataset: augmentation OFF, cache ON, prefetch ON")

# Test Dataset
# ⬇️ PERUBAHAN: Menambahkan .cache().prefetch() ⬇️
test_ds = test_ds.map(
    lambda x, y: preprocess_for_efficientnet(x, y, training=False),
    num_parallel_calls=AUTOTUNE
).cache().prefetch(buffer_size=AUTOTUNE)

print("✓ Test dataset: augmentation OFF, cache ON, prefetch ON")

print("\n" + "="*60)
print("PREPROCESSING SELESAI")
print("="*60)
print("\nDataset siap digunakan untuk training EfficientNet-B0!")

In [ ]:
# =============================================================================
# CELL 7: VERIFIKASI PREPROCESSING
# =============================================================================
# Cell ini memverifikasi bahwa preprocessing EfficientNet bekerja dengan benar
# Range nilai yang diharapkan: sekitar [-2.1, 2.6] (berbeda dengan ResNet50!)
# =============================================================================

print("Verifikasi preprocessing...")

for images, labels in train_ds.take(1):
    print(f"\nBatch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"\nStatistik pixel setelah preprocessing:")
    print(f"  - Min: {tf.reduce_min(images).numpy():.4f}")
    print(f"  - Max: {tf.reduce_max(images).numpy():.4f}")
    print(f"  - Mean: {tf.reduce_mean(images).numpy():.4f}")
    print(f"  - Std: {tf.math.reduce_std(images).numpy():.4f}")
    
    min_val = tf.reduce_min(images).numpy()
    max_val = tf.reduce_max(images).numpy()
    
    # EfficientNet preprocess_input menghasilkan nilai dalam range sekitar [-2.1, 2.6]
    # (berbeda dengan ResNet50 yang ~[-128, 128])
    if -3 < min_val < 0 and 0 < max_val < 3:
        print("\n✅ Range nilai sesuai dengan ekspektasi EfficientNet")
        print("   (EfficientNet: ~[-2.1, 2.6], ResNet50: ~[-128, 128])")
    elif min_val < -50 or max_val > 50:
        print("\n⚠️ WARNING: Range nilai terlihat seperti ResNet50!")
        print("   Pastikan import dari efficientnet, bukan resnet50")
    else:
        print("\n⚠️ Range nilai tidak sesuai ekspektasi")

---

## 📋 RINGKASAN PERUBAHAN

| Cell | Perubahan | Detail |
|------|-----------|--------|
| **Cell 1** | ✅ Import berbeda | `efficientnet` bukan `resnet50` |
| **Cell 2** | ❌ Tidak ada | Konfigurasi sama |
| **Cell 3** | ❌ Tidak ada | Load dataset sama |
| **Cell 4** | ❌ Tidak ada | Augmentasi **TETAP SAMA** (hanya flip & rotation) |
| **Cell 5** | ✅ Perilaku berbeda | `preprocess_input()` adalah versi EfficientNet |
| **Cell 6** | ✅ Optimasi pipeline | Menambahkan `.cache()` dan `.prefetch()` |
| **Cell 7** | ✅ Verifikasi berbeda | Cek range EfficientNet vs ResNet50 |

---